In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import re 
import matplotlib.pyplot as plt 


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format',lambda x: '%.3f' % x)

In [7]:
#Reading Dataset 

In [3]:
df_ = pd.read_csv("flo_data_20k.csv")
df = df_.copy()

In [4]:
#Step 2 
#First 10 observation : 
df.head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR]
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-11-18,2021-03-13,2018-11-18,2021-03-13,1.000,2.000,150.870,49.990,[KADIN]
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,Android App,Android App,2020-03-04,2020-10-18,2020-10-18,2020-03-04,3.000,1.000,59.990,315.940,[AKTIFSPOR]
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,Mobile,Offline,2020-05-15,2020-08-12,2020-05-15,2020-08-12,1.000,1.000,49.990,113.640,[COCUK]
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,Android App,Android App,2020-01-23,2021-03-07,2021-03-07,2020-01-25,3.000,2.000,120.480,934.210,"[ERKEK, COCUK, KADIN]"
9,1143f032-440d-11ea-8b43-000d3a38a36f,Mobile,Mobile,2019-07-30,2020-10-04,2020-10-04,2019-07-30,1.000,1.000,69.980,95.980,"[KADIN, AKTIFSPOR]"


In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.000,3.111,4.226,1.000,1.000,2.000,4.000,200.000
order_num_total_ever_offline,19945.000,1.914,2.063,1.000,1.000,1.000,2.000,109.000
customer_value_total_ever_offline,19945.000,253.923,301.533,10.000,99.990,179.980,319.970,18119.140
customer_value_total_ever_online,19945.000,497.322,832.602,12.990,149.980,286.460,578.440,45220.130


In [12]:
df.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12'],
      dtype='object')

In [13]:
df.shape

(19945, 12)

In [16]:
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

In [17]:
df.dtypes

master_id                             object
order_channel                         object
last_order_channel                    object
first_order_date                      object
last_order_date                       object
last_order_date_online                object
last_order_date_offline               object
order_num_total_ever_online          float64
order_num_total_ever_offline         float64
customer_value_total_ever_offline    float64
customer_value_total_ever_online     float64
interested_in_categories_12           object
dtype: object

In [21]:
#Total for each customer number of purchases and spending.
#Total total_transaction = order_num_total_ever_online + order_num_total_ever_offline
#Total Price for each customer: (order_num_total_ever_online * customer_value_total_ever_online) + (order_num_total_ever_offline*customer_value_total_ever_offline)
df["total_price"] =  (df["order_num_total_ever_online"]*df["customer_value_total_ever_online"]) + (df["order_num_total_ever_offline"]*df["customer_value_total_ever_offline"])
df["total_transaction"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]

In [22]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_price,total_transaction
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN],3337.510,5.000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",35537.960,21.000
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]",1565.990,5.000
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]",121.970,2.000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR],209.980,2.000


In [23]:
#Converting String types to date type.
df["first_order_date"] = pd.to_datetime(df["first_order_date"])
df["last_order_date"] = pd.to_datetime(df["last_order_date"])
df["last_order_date_online"] = pd.to_datetime(df["last_order_date_online"])
df["last_order_date_offline"] = pd.to_datetime(df["last_order_date_offline"])

In [24]:
df.dtypes

master_id                                    object
order_channel                                object
last_order_channel                           object
first_order_date                     datetime64[ns]
last_order_date                      datetime64[ns]
last_order_date_online               datetime64[ns]
last_order_date_offline              datetime64[ns]
order_num_total_ever_online                 float64
order_num_total_ever_offline                float64
customer_value_total_ever_offline           float64
customer_value_total_ever_online            float64
interested_in_categories_12                  object
total_price                                 float64
total_transaction                           float64
dtype: object

In [25]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.000,3.111,4.226,1.000,1.000,2.000,4.000,200.000
order_num_total_ever_offline,19945.000,1.914,2.063,1.000,1.000,1.000,2.000,109.000
customer_value_total_ever_offline,19945.000,253.923,301.533,10.000,99.990,179.980,319.970,18119.140
customer_value_total_ever_online,19945.000,497.322,832.602,12.990,149.980,286.460,578.440,45220.130
total_price,19945.000,5334.164,73015.488,44.980,469.890,1144.780,3062.980,9045395.940
total_transaction,19945.000,5.025,4.743,2.000,3.000,4.000,6.000,202.000


In [38]:
#Top 10 customers with the most profits
df.sort_values("total_price", ascending = False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_price,total_transaction
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.000,2.000,684.970,45220.130,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",9045395.940,202.000
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.000,1.000,1263.760,32654.340,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",2646265.300,82.000
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.000,1.000,130.490,36687.800,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",2458213.090,68.000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.000,1.000,49.990,12232.250,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",1590242.490,131.000
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.000,4.000,843.680,17599.890,"[ERKEK, KADIN, AKTIFSPOR]",1164967.460,70.000
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.000,1.000,82.480,16836.090,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",1161772.690,70.000
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.000,109.000,10239.460,143.980,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",1116389.100,111.000
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.000,87.000,8432.250,139.980,[ERKEK],733745.730,88.000
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.000,1.000,180.730,12545.370,"[ERKEK, AKTIFSPOR]",451814.050,37.000
6337,f7e2335e-a7c4-11e9-a2fc-000d3a38a36f,Ios App,Mobile,2015-11-23,2020-08-08,2020-08-08,2020-06-26,54.000,7.000,820.330,7133.140,"[COCUK, KADIN]",390931.870,61.000


In [39]:
df.sort_values("total_transaction", ascending = False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_price,total_transaction
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.000,2.000,684.970,45220.130,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",9045395.940,202.000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.000,1.000,49.990,12232.250,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",1590242.490,131.000
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.000,109.000,10239.460,143.980,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",1116389.100,111.000
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.000,87.000,8432.250,139.980,[ERKEK],733745.730,88.000
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.000,81.000,3997.550,242.810,[ERKEK],324287.170,83.000
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.000,1.000,1263.760,32654.340,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",2646265.300,82.000
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.000,74.000,4640.770,543.280,"[KADIN, AKTIFSPOR]",345046.820,77.000
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.000,3.000,292.930,5004.950,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",361235.190,75.000
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.000,1.000,82.480,16836.090,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",1161772.690,70.000
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.000,4.000,843.680,17599.890,"[ERKEK, KADIN, AKTIFSPOR]",1164967.460,70.000


In [9]:
def data_preparation(dataframe):
    #Total for each customer number of purchases and spending.
    #Total total_transaction = order_num_total_ever_online + order_num_total_ever_offline
    #Total Price for each customer: (order_num_total_ever_online * customer_value_total_ever_online) + (order_num_total_ever_offline*customer_value_total_ever_offline)
    df["total_price"] =  (df["order_num_total_ever_online"]*df["customer_value_total_ever_online"]) + (df["order_num_total_ever_offline"]*df["customer_value_total_ever_offline"])
    df["total_transaction"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
    
    #Converting String types to date type.
    df["first_order_date"] = pd.to_datetime(df["first_order_date"])
    df["last_order_date"] = pd.to_datetime(df["last_order_date"])
    df["last_order_date_online"] = pd.to_datetime(df["last_order_date_online"])
    df["last_order_date_offline"] = pd.to_datetime(df["last_order_date_offline"])
    return df
    

In [10]:
data_preparation(df)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,total_price,total_transaction
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.000,1.000,139.990,799.380,[KADIN],3337.510,5.000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.000,2.000,159.970,1853.580,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",35537.960,21.000
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.000,2.000,189.970,395.350,"[ERKEK, KADIN]",1565.990,5.000
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.000,1.000,39.990,81.980,"[AKTIFCOCUK, COCUK]",121.970,2.000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.000,1.000,49.990,159.990,[AKTIFSPOR],209.980,2.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,Android App,Offline,2019-09-21,2020-07-05,2020-06-05,2020-07-05,1.000,2.000,289.980,111.980,"[ERKEK, AKTIFSPOR]",691.940,3.000
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,Desktop,Desktop,2020-03-01,2020-12-22,2020-12-22,2020-03-01,1.000,1.000,150.480,239.990,[AKTIFSPOR],390.470,2.000
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,Ios App,Ios App,2019-09-11,2021-05-24,2021-05-24,2019-09-11,2.000,1.000,139.980,492.960,[AKTIFSPOR],1125.900,3.000
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,Android App,Android App,2019-03-27,2021-02-13,2021-02-13,2021-01-08,1.000,5.000,711.790,297.980,"[ERKEK, AKTIFSPOR]",3856.930,6.000


In [13]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [21]:
today_date = dt.datetime(2021,6,1)

In [12]:
#Recency, Frequency and Monetary nomination: 

# Recency = The "Recency" value is an indicator calculated on a daily basis, showing when the customer last contacted the company.
# Frequency = Frequency is a measure of how many times the customer has financial interactions with the firm.
# Monetary = Monetary value is the total income that the customer has brought to the company.

In [75]:
rfm = df.groupby("master_id").agg({"last_order_date": lambda date :(today_date - date),
                                   "total_transaction": lambda x : x,
                                   "total_price": lambda x: x,"interested_in_categories_12":lambda x: x}) 

In [76]:
rfm

,last_order_date,total_transaction,total_price,interested_in_categories_12
master_id,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10 days,5.000,1979.330,"[ERKEK, COCUK, AKTIFSPOR]"
00034aaa-a838-11e9-a2fc-000d3a38a36f,298 days,3.000,419.950,"[ERKEK, KADIN]"
000be838-85df-11ea-a90b-000d3a38a36f,213 days,4.000,1436.630,"[AKTIFCOCUK, AKTIFSPOR]"
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27 days,7.000,3153.670,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]"
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20 days,7.000,7201.710,"[ERKEK, AKTIFSPOR]"
...,...,...,...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,183 days,5.000,2347.720,[AKTIFSPOR]
fff4736a-60a4-11ea-8dd8-000d3a38a36f,67 days,2.000,318.870,[]
fffacd34-ae14-11e9-a2fc-000d3a38a36f,257 days,6.000,4679.680,[ERKEK]


In [78]:
rfm.columns = ["recency","frequency","monetary","interested_cats"]

In [79]:
rfm.reset_index

<bound method DataFrame.reset_index of                                       recency  frequency  monetary  \
master_id                                                            
00016786-2f5a-11ea-bb80-000d3a38a36f  10 days      5.000  1979.330   
00034aaa-a838-11e9-a2fc-000d3a38a36f 298 days      3.000   419.950   
000be838-85df-11ea-a90b-000d3a38a36f 213 days      4.000  1436.630   
000c1fe2-a8b7-11ea-8479-000d3a38a36f  27 days      7.000  3153.670   
000f5e3e-9dde-11ea-80cd-000d3a38a36f  20 days      7.000  7201.710   
...                                       ...        ...       ...   
fff1db94-afd9-11ea-b736-000d3a38a36f 183 days      5.000  2347.720   
fff4736a-60a4-11ea-8dd8-000d3a38a36f  67 days      2.000   318.870   
fffacd34-ae14-11e9-a2fc-000d3a38a36f 257 days      6.000  4679.680   
fffacecc-ddc3-11e9-a848-000d3a38a36f 132 days      9.000  6331.590   
fffe4b30-18e0-11ea-9213-000d3a38a36f 321 days      4.000  1049.380   

                                                  

In [80]:
rfm["recency_score"] = pd.qcut(rfm["recency"],5,labels = [5,4,3,2,1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method = "first"), 5, labels = [1,2,3,4,5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels = [1,2,3,4,5])

In [81]:
rfm

,recency,frequency,monetary,interested_cats,recency_score,frequency_score,monetary_score
master_id,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10 days,5.000,1979.330,"[ERKEK, COCUK, AKTIFSPOR]",5,4,4
00034aaa-a838-11e9-a2fc-000d3a38a36f,298 days,3.000,419.950,"[ERKEK, KADIN]",1,2,2
000be838-85df-11ea-a90b-000d3a38a36f,213 days,4.000,1436.630,"[AKTIFCOCUK, AKTIFSPOR]",2,3,3
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27 days,7.000,3153.670,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",5,4,4
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20 days,7.000,7201.710,"[ERKEK, AKTIFSPOR]",5,4,5
...,...,...,...,...,...,...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,183 days,5.000,2347.720,[AKTIFSPOR],2,4,4
fff4736a-60a4-11ea-8dd8-000d3a38a36f,67 days,2.000,318.870,[],4,2,1
fffacd34-ae14-11e9-a2fc-000d3a38a36f,257 days,6.000,4679.680,[ERKEK],1,4,5


In [82]:
rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))

In [83]:
rfm

,recency,frequency,monetary,interested_cats,recency_score,frequency_score,monetary_score,RF_SCORE
master_id,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10 days,5.000,1979.330,"[ERKEK, COCUK, AKTIFSPOR]",5,4,4,54
00034aaa-a838-11e9-a2fc-000d3a38a36f,298 days,3.000,419.950,"[ERKEK, KADIN]",1,2,2,12
000be838-85df-11ea-a90b-000d3a38a36f,213 days,4.000,1436.630,"[AKTIFCOCUK, AKTIFSPOR]",2,3,3,23
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27 days,7.000,3153.670,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",5,4,4,54
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20 days,7.000,7201.710,"[ERKEK, AKTIFSPOR]",5,4,5,54
...,...,...,...,...,...,...,...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,183 days,5.000,2347.720,[AKTIFSPOR],2,4,4,24
fff4736a-60a4-11ea-8dd8-000d3a38a36f,67 days,2.000,318.870,[],4,2,1,42
fffacd34-ae14-11e9-a2fc-000d3a38a36f,257 days,6.000,4679.680,[ERKEK],1,4,5,14


In [84]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [85]:
rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex = True)

In [86]:
rfm

,recency,frequency,monetary,interested_cats,recency_score,frequency_score,monetary_score,RF_SCORE,segment
master_id,,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10 days,5.000,1979.330,"[ERKEK, COCUK, AKTIFSPOR]",5,4,4,54,champions
00034aaa-a838-11e9-a2fc-000d3a38a36f,298 days,3.000,419.950,"[ERKEK, KADIN]",1,2,2,12,hibernating
000be838-85df-11ea-a90b-000d3a38a36f,213 days,4.000,1436.630,"[AKTIFCOCUK, AKTIFSPOR]",2,3,3,23,at_Risk
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27 days,7.000,3153.670,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",5,4,4,54,champions
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20 days,7.000,7201.710,"[ERKEK, AKTIFSPOR]",5,4,5,54,champions
...,...,...,...,...,...,...,...,...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,183 days,5.000,2347.720,[AKTIFSPOR],2,4,4,24,at_Risk
fff4736a-60a4-11ea-8dd8-000d3a38a36f,67 days,2.000,318.870,[],4,2,1,42,potential_loyalists
fffacd34-ae14-11e9-a2fc-000d3a38a36f,257 days,6.000,4679.680,[ERKEK],1,4,5,14,at_Risk


In [87]:
rfm[["segment", "recency", "frequency","monetary"]].groupby("segment").agg(["mean","count"])

recency       frequency        \
                                           mean count      mean count   
segment                                                                 
about_to_sleep      114 days 00:45:34.510042604  1643     2.407  1643   
at_Risk             242 days 07:53:45.380710660  3152     4.470  3152   
cant_loose          235 days 03:49:08.743718592  1194    10.717  1194   
champions                      17 days 03:24:45  1920     8.965  1920   
hibernating         247 days 10:13:52.543884092  3589     2.391  3589   
loyal_customers         82 days 13:23:24.800000  3375     8.356  3375   
need_attention      113 days 00:53:35.880893300   806     3.739   806   
new_customers        17 days 23:25:45.913818722   673     2.000   673   
potential_loyalists  36 days 20:52:25.846153846  2925     3.311  2925   
promising            58 days 16:40:14.371257485   668     2.000   668   

                     monetary        
                         mean count  
segment                              
about_to_sleep        481.625  1643  
at_Risk              1829.340  3152  
cant_loose          16088.820  1194  
champions           19499.284  1920  
hibernating           480.075  3589  
loyal_customers     10954.101  3375  
need_attention       1211.415   806  
new_customers         344.049   673  
potential_loyalists  1046.015  2925  
promising             334.153   668

In [88]:
def rfm_score_producer(dataframe):
    df["last_order_date"].max()
    today_date = dt.datetime(2021,6,1)
    
    rfm = df.groupby("master_id").agg({"last_order_date": lambda date :(today_date - date),
                                   "total_transaction": lambda x : x,
                                   "total_price": lambda x: x}) 
    rfm.columns = ["recency","frequency","monetary"]
    rfm["recency_score"] = pd.qcut(rfm["recency"],5,labels = [5,4,3,2,1])
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method = "first"), 5, labels = [1,2,3,4,5])
    rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels = [1,2,3,4,5])
    
    rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))
    
    
    seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
    }
    
    rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex = True)
    
    return rfm
    
    

In [90]:
rfm

,recency,frequency,monetary,interested_cats,recency_score,frequency_score,monetary_score,RF_SCORE,segment
master_id,,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10 days,5.000,1979.330,"[ERKEK, COCUK, AKTIFSPOR]",5,4,4,54,champions
00034aaa-a838-11e9-a2fc-000d3a38a36f,298 days,3.000,419.950,"[ERKEK, KADIN]",1,2,2,12,hibernating
000be838-85df-11ea-a90b-000d3a38a36f,213 days,4.000,1436.630,"[AKTIFCOCUK, AKTIFSPOR]",2,3,3,23,at_Risk
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27 days,7.000,3153.670,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",5,4,4,54,champions
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20 days,7.000,7201.710,"[ERKEK, AKTIFSPOR]",5,4,5,54,champions
...,...,...,...,...,...,...,...,...,...
fff1db94-afd9-11ea-b736-000d3a38a36f,183 days,5.000,2347.720,[AKTIFSPOR],2,4,4,24,at_Risk
fff4736a-60a4-11ea-8dd8-000d3a38a36f,67 days,2.000,318.870,[],4,2,1,42,potential_loyalists
fffacd34-ae14-11e9-a2fc-000d3a38a36f,257 days,6.000,4679.680,[ERKEK],1,4,5,14,at_Risk


In [91]:
#a. FLO includes a new women's shoe brand. Product prices of the included brand
#on your preferences. For this reason, it is specially designed with customers in the profile that will be interested in the promotion of the brand and product sales.
#you want to contact. Shopping from loyal customers (champions, loyal_customers) and female category
#customers to be contacted specifically. Save the id numbers of these customers to the csv file.

In [106]:
type_a_df = rfm.loc[(rfm["segment"] == "champions") | (rfm["segment"] == "loyal_customers") & rfm["interested_cats"].str.contains("KADIN")].reset_index()

In [108]:
type_a_df["master_id"].to_csv('type_a_customers.csv')

In [111]:
type_b_df = rfm.loc[(rfm["segment"] == "cant_loose")  & (rfm["interested_cats"].str.contains("ERKEK")) & (rfm["interested_cats"].str.contains("COCUK")) ].reset_index()
type_b_df

,master_id,recency,frequency,monetary,interested_cats,recency_score,frequency_score,monetary_score,RF_SCORE,segment
0,08ce95e8-aac9-11e9-a2fc-000d3a38a36f,237 days,17.000,42250.080,"[AKTIFCOCUK, ERKEK, COCUK, AKTIFSPOR]",1,5,5,15,cant_loose
1,0a4e6440-a9dc-11e9-a2fc-000d3a38a36f,273 days,14.000,28111.010,"[AKTIFCOCUK, ERKEK, COCUK]",1,5,5,15,cant_loose
2,0d9a4224-ab65-11e9-a2fc-000d3a38a36f,166 days,16.000,52482.550,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",2,5,5,25,cant_loose
3,0f37c9fc-26c6-11eb-8a9b-000d3a38a36f,162 days,10.000,4819.280,"[AKTIFCOCUK, ERKEK, AKTIFSPOR]",2,5,5,25,cant_loose
4,10f7164e-a8c3-11e9-a2fc-000d3a38a36f,186 days,8.000,4672.240,"[AKTIFCOCUK, ERKEK, COCUK, AKTIFSPOR]",2,5,5,25,cant_loose
...,...,...,...,...,...,...,...,...,...,...
187,f81789aa-ad05-11e9-a2fc-000d3a38a36f,181 days,10.000,7285.650,"[ERKEK, COCUK, AKTIFSPOR]",2,5,5,25,cant_loose
188,f88cf3ea-47f4-11ea-b6c0-000d3a38a36f,152 days,7.000,9348.820,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",2,5,5,25,cant_loose
189,f8b04ba4-5bce-11ea-ba31-000d3a38a36f,196 days,7.000,5868.970,"[AKTIFCOCUK, ERKEK, KADIN]",2,5,5,25,cant_loose
190,f91f4b9e-dd5b-11ea-9c1f-000d3a38a36f,292 days,9.000,5851.650,"[ERKEK, COCUK, KADIN]",1,5,5,15,cant_loose


In [112]:
type_b_df["master_id"].to_csv('type_b_customers.csv')